In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

%matplotlib inline

In [24]:
plt.rc('figure',dpi=100);

In [25]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [26]:
vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

vistas.head()

,idaviso,idpostulante,cantidad,total_vistas
0,18,BolNL,2,28
1,1738407,BolNL,1,28
2,1795995,BolNL,2,28
3,1804592,BolNL,1,28
4,1804723,BolNL,1,28


In [27]:
#seteo el peso como cantidad de vistas del postulante al anuncio/total de vistas del postulante
vistas["cantidad"] = vistas["cantidad"]/vistas["total_vistas"]
vistas = vistas.drop("total_vistas",axis=1)
vistas.head()


,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,1738407,BolNL,0.035714
2,1795995,BolNL,0.071429
3,1804592,BolNL,0.035714
4,1804723,BolNL,0.035714


In [28]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
no_postulaciones=pd.merge(no_postulaciones,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
no_postulaciones=pd.merge(no_postulaciones,avisos,on='idaviso')

no_postulaciones.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,graduados,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social
0,1112430536,EexaOz,0,MASC,38.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,False
1,1112430536,owaWw0Z,0,FEM,20.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False
2,1112430536,PmGPRd1,0,MASC,19.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False
3,1112430536,bO4q324,0,MASC,25.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False
4,1112430536,MVd5dlr,0,MASC,30.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,False


In [29]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,graduados,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,True,False
1,1112257047,1kJqGb,1,FEM,34.0,-,-,-,-,-,...,False,False,False,False,True,True,False,False,True,False
2,1112257047,eOE9Rr,1,FEM,39.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,True,False
3,1112257047,Zrx8Xz,1,FEM,39.0,-,-,-,-,-,...,False,False,False,False,True,True,False,False,True,False
4,1112257047,ZrKNQY,1,FEM,32.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,True,False


In [30]:
postulaciones=postulaciones.append(no_postulaciones, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)

postulaciones = postulaciones.sample(len(postulaciones)//2)
print(len(postulaciones))

12793168
6393793


In [31]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social,cantidad
5097585,1112437073,ekNkZM2,1,FEM,26.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.001300
2730939,1112278863,JBr8b5k,0,FEM,47.0,-,-,-,-,-,...,False,False,False,True,False,False,False,False,False,NaN
3764677,1112424076,2zP0K36,1,MASC,20.0,-,-,Graduado,-,Graduado,...,False,False,False,False,True,False,False,False,False,NaN
2389152,1112478377,ZD2bxXR,0,MASC,46.0,-,-,-,-,Abandonado,...,False,False,False,False,False,False,False,False,False,NaN
6372937,1112446878,8MZA0Q3,0,FEM,50.0,-,-,-,-,-,...,False,False,False,False,True,False,False,False,False,NaN
6381601,1112447386,JBxxjWp,1,FEM,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
1952521,1002101668,8M2w5eL,1,FEM,23.0,-,-,-,-,Graduado,...,False,False,True,False,True,False,False,False,True,NaN
826238,1112094756,JBxBEL3,1,FEM,22.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,False,False,NaN
5010120,1112430991,VNP0R1j,0,MASC,20.0,-,-,Graduado,-,Graduado,...,False,False,True,False,False,False,False,False,False,NaN
2688016,1112370276,JBxDPKJ,0,FEM,22.0,-,-,-,-,-,...,False,False,False,False,False,False,True,False,False,NaN


In [32]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
postulaciones2=0

Train:  5754413 Test:  639380


In [33]:
import gc
gc.collect()

39

In [34]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=300,max_depth=9,n_estimators=1000,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [35]:
score=c.score(x_test,y_test)*100
print(score)

78.00509868935531


C:\Users\Pocho\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
joblib.dump(c, "Data/fiuba_entrenamiento/pocho/modelos/mini-LGBMV2.pkl")

['Data/fiuba_entrenamiento/pocho/modelos/mini-LGBMV2.pkl']

In [37]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social,cantidad
51362,77759,1112464431,GNJDObp,MASC,27.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.060606
28974,68890,1112460932,vVDVJKe,FEM,52.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,True,False,False,0.001277
95475,94914,1112486711,Ep3pK2,MASC,38.0,-,-,-,En Curso,Graduado,...,True,False,False,True,True,False,True,False,False,0.000000
36636,85562,1112466646,jkV55b6,FEM,26.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,0.006329
27543,39735,1112417833,6rYwvx,FEM,35.0,-,-,-,Graduado,Graduado,...,False,False,True,True,False,False,False,False,False,0.002558
16102,68738,1112460842,mzb5qOQ,FEM,26.0,-,-,-,-,Graduado,...,False,False,True,False,True,False,True,False,False,0.019608
54040,68406,1112460708,1Y4l6j,MASC,32.0,-,-,-,-,-,...,True,False,False,False,False,False,False,False,False,0.000000
79240,26773,1112344850,ZYmqvZ,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
38908,70352,1112461730,GNX3b8v,MASC,21.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,True,0.004630
69547,15122,1112118055,1Qmzvb,MASC,38.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000


In [38]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.7044177 , 0.2955823 ],
       [0.69145526, 0.30854474],
       [0.69533838, 0.30466162],
       ...,
       [0.8042222 , 0.1957778 ],
       [0.63292659, 0.36707341],
       [0.93316297, 0.06683703]])

In [39]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [40]:
prediccion['sepostulo']=y_final2

In [41]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.295582
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.308545
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.304662
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.116158
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.069422


In [42]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.295582
1,1,0.308545
2,2,0.304662
3,3,0.116158
4,4,0.069422


In [43]:
len(prediccion)

100000

In [44]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/pred-minilgbm2-poch.csv",index=False)
prediccion.sepostulo.mean()

0.5795455432042231